## **Utilisation simple des identifieurs de langues** (rev2) **avec les :**
>### **- '*Sparse*' Bag Of Words**
>### **- Tokenisations BERT ou Tiktoken**
>### **- CountVectorizer utilisant une tokenisation '*custom*'**
>### **- Classificateurs Naïve Bayes et Gradiant Boosting**

#### **Choix du tokenizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import joblib
from sklearn.feature_extraction.text import CountVectorizer

# Choix de la Tokenisation (False = BERT, True Tiktoken)
titoken_tokenization = True

# Ce parametre permet éventuellement d'équilibrer de nombre de phrase par langue.
# Si ce parametre est très grand, tout le corpus sera lu. 
nb_phrase_lang = 500000

import warnings
warnings.filterwarnings('ignore')

#### **Lectures des phrases de "sentences.csv", et de leur étiquette "Langue" pour les langues sélectionnées**

In [2]:
# Ouvrir le fichier d'entrée en mode lecture
def create_lang_df(path):
    df = pd.read_csv(path, index_col ='id')
    return df

df_big = create_lang_df('../data/multilingue/sentences.csv')
lan_code = ['eng','fra','deu','spa','ita']
df = pd.DataFrame(columns=df_big.columns)
for i in range(len(lan_code)):
    df= pd.concat([df, df_big[df_big['lan_code']==lan_code[i]].iloc[:nb_phrase_lang]])
df = df.sample(frac=1, random_state=3).reset_index(drop=True)
n_rows = len(df)
print('Nombre de lignes de sentence.csv:',n_rows)
nb_phrases_lang =[]
for l in lan_code:
    nb_phrases_lang.append(sum(df['lan_code']==l))
print("Nombre de phrases par langue ",lan_code,":",nb_phrases_lang)
display(df.head(10))
display(df.tail(10))

Nombre de lignes de sentence.csv: 1750000
Nombre de phrases par langue  ['eng', 'fra', 'deu', 'spa', 'ita'] : [350000, 350000, 350000, 350000, 350000]


,lan_code,sentence
0,eng,She is afraid of death.
1,ita,Indovina cosa scelgo io.
2,spa,"¿Puedo ayudarlo? ""No, gracias. Solo estoy mira..."
3,ita,Io non sono una fricchettona!
4,deu,"Es sind schon fast 10 Jahre vergangen, aber du..."
5,spa,Creía que me quería.
6,eng,This school sets high moral standards for pupils.
7,eng,"Man is judged by his courage, woman by her charm."
8,fra,Je mange des pruneaux sucrés.
9,fra,J'ai écrit une chanson pour toi.


,lan_code,sentence
1749990,deu,Es geschieht heutzutage ja so viel in unserer ...
1749991,spa,El almuerzo está preparado.
1749992,eng,I've seen enough.
1749993,ita,Hanno accelerato il passo.
1749994,fra,Elle en pince pour ce garçon.
1749995,deu,"Wer von uns wünschte nicht manchmal, dass er d..."
1749996,ita,No! Io odio i broccoli!
1749997,fra,Tu seras tuée !
1749998,fra,Tom aurait dû manger plus.
1749999,eng,He took the video to a local TV station.


#### **Selection du tokenizer** en fonction de la variable titoken_tokenization <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [3]:
# Selection du tokenizer
if titoken_tokenization:
    import tiktoken
    tokenizer = tiktoken.get_encoding("cl100k_base")
else:
    from transformers import BertTokenizerFast
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-uncased')

#### **Préparation de la vectorisation par CountVectorizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>
#### **Création de la fonction de création d'un Bags  Of Worlds pour les phrases à identifier** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [4]:
# Les 2 fonctions suivantes sont nécéssaires afin de sérialiser ces parametre de CountVectorizer
# et ainsi de sauvegarder le vectorizer pour un un usage ultérieur sans utiliser X_train pour  le réinitialiser
def custom_tokenizer(text):
    tokens = tokenizer.encode(text)  # Cela divise le texte en mots
    return tokens

def custom_preprocessor(text):
    return text

# CountVectorizer a une liste de phrase en entrée.
# Cette fonction met les données d'entrée dans le bon format
def format_to_vectorize(data):
    X_tok = []
    if "DataFrame" in str(type(data)):sentences = df.tolist()
    elif "str" in str(type(data)):
        sentences =[data]
    else: sentences = data
                          
    for sentence in sentences:
        X_tok.append(sentence) # ('¤'.join([tokenizer.decode([ids]) for ids in tokenizer.encode(sentence)])+'¤')
    return X_tok

def create_BOW(data):
    global vectorizer
    
    X_tok = format_to_vectorize(data)
    X = vectorizer.transform(X_tok)
    return X

#### **Chargement du vectorizer** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [5]:
def load_vectorizer():
    global dict_token, dict_ids, nb_token
    
    if titoken_tokenization: path = '../data/vectorizer_tiktoken.pkl'
    else: path = '../data/vectorizer_BERT.pkl'
    vectorizer = joblib.load(path)
    dict_token = {tokenizer.decode([cle]): cle for cle, valeur in vectorizer.vocabulary_.items()}
    dict_ids = {cle: tokenizer.decode([cle]) for cle, valeur in vectorizer.vocabulary_.items()} #dict_ids.items()}
    nb_token = len(vectorizer.vocabulary_)
    return vectorizer

vectorizer = load_vectorizer()

#### **Choix du nom du fichier du classifieur sauvegardé** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [6]:
def get_file_name(titoken_tokenization, classifier):
    if titoken_tokenization:
        return "id_lang_tiktoken_"+classifier+"_sparse.pkl"
    else:
        return "id_lang_BERT_"+classifier+"_sparse.pkl"

#### **Chargement du classificateur entrainé avec l'algorithme Naïve Bayes** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [7]:
from sklearn import naive_bayes

# Chargement du classificateur sauvé
clf_nb = joblib.load("../data/"+get_file_name(titoken_tokenization,"nb"))


#### **Chargement du classificateur entrainé avec l'algorithme Gradiant Boosting**

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

# Chargement du classificateur sauvé
clf_gb = joblib.load("../data/"+get_file_name(titoken_tokenization,"gb"))
######### dict_ids, decoded_keys = load_dict_token() ######### 

#### **Definition de fonctions identificateur de langue avec Naive Bayes** <font color='red'>(nécéssaire pour traduction texte libre)</font>

In [9]:
import json

# Chargement du classificateur sauvegardé
clf_nb = joblib.load("../data/"+get_file_name(titoken_tokenization,"nb"))
vectorizer = load_vectorizer()

# Lisez le contenu du fichier JSON
with open('../data/multilingue/lan_to_language.json', 'r') as fichier:
    lan_to_language = json.load(fichier)



def lang_id_nb(sentences):
    if "str" in str(type(sentences)):
        return lan_to_language[clf_nb.predict(create_BOW(sentences))[0]]
    else: return [lan_to_language[l] for l in clf_nb.predict(create_BOW(sentences))]

In [10]:
def lang_id_gb(sentences):
    if "str" in str(type(sentences)):
        return lan_to_language[clf_gb.predict(create_BOW(sentences))[0]]
    else:
        return [lan_to_language[l] for l in clf_gb.predict(create_BOW(sentences))]        

#### **Exemples d'utilisation**

In [11]:
import random
# Instanciation d'exemples

sentence_no = random.sample(range(len(df)),5)

exemples = ["france is often snowy during spring , and it is relaxing in january .",
           "elle adore les voitures très luxueuses, et toi ?",
           "she loves very luxurious cars, don't you?",
           "vamos a la playa",
           "Ich heiße Keyne, und das ist wunderbar",
           "she loves you, mais elle te hait aussi, and das ist traurig", # Attention à cette phrase trilingue
           "I ate caviar", 
           "Vogliamo visitare il Colosseo e nuotare nel Tevere.",
            df['sentence'].iloc[sentence_no[0]],
            df['sentence'].iloc[sentence_no[1]],
            df['sentence'].iloc[sentence_no[2]],
            df['sentence'].iloc[sentence_no[3]],
            df['sentence'].iloc[sentence_no[4]],
          ]
lang_exemples = ['eng','fra','eng','spa','deu','e,f,d','en','ita',df['lan_code'].iloc[sentence_no[0]],df['lan_code'].iloc[sentence_no[1]],df['lan_code'].iloc[sentence_no[2]],
                 df['lan_code'].iloc[sentence_no[3]],df['lan_code'].iloc[sentence_no[4]]]
print('no\t lang\t Phrase')                            
for i in range(len(exemples)):
    print(i,'-\t',lang_exemples[i],'\t',exemples[i])

no	 lang	 Phrase
0 -	 eng 	 france is often snowy during spring , and it is relaxing in january .
1 -	 fra 	 elle adore les voitures très luxueuses, et toi ?
2 -	 eng 	 she loves very luxurious cars, don't you?
3 -	 spa 	 vamos a la playa
4 -	 deu 	 Ich heiße Keyne, und das ist wunderbar
5 -	 e,f,d 	 she loves you, mais elle te hait aussi, and das ist traurig
6 -	 en 	 I ate caviar
7 -	 ita 	 Vogliamo visitare il Colosseo e nuotare nel Tevere.
8 -	 fra 	 Il ne me plaît pas, mais elle, elle me plaît.
9 -	 eng 	 Mr Sato ran a supermarket in his hometown before he came to Tokyo.
10 -	 deu 	 Er erwarb die amerikanische Staatsbürgerschaft.
11 -	 spa 	 Yo ocupo todo el tiempo que puedo en leer.
12 -	 eng 	 His life rests on her.


In [12]:
# Affichage des prédictions
print("Langue réelle\tPréd. Naive B.\tPréd. Grad. B.\tPhrase")
for i in range(len(exemples)):
    print(lang_exemples[i]+'\t\t'+lang_id_nb(exemples[i])+'\t\t'+lang_id_gb(exemples[i])+'\t\t'+exemples[i])


Langue réelle	Préd. Naive B.	Préd. Grad. B.	Phrase
eng		English		English		france is often snowy during spring , and it is relaxing in january .
fra		French		French		elle adore les voitures très luxueuses, et toi ?
eng		English		English		she loves very luxurious cars, don't you?
spa		Spanish		Spanish		vamos a la playa
deu		German		German		Ich heiße Keyne, und das ist wunderbar
e,f,d		French		German		she loves you, mais elle te hait aussi, and das ist traurig
en		Italian		English		I ate caviar
ita		Italian		Italian		Vogliamo visitare il Colosseo e nuotare nel Tevere.
fra		French		French		Il ne me plaît pas, mais elle, elle me plaît.
eng		English		English		Mr Sato ran a supermarket in his hometown before he came to Tokyo.
deu		German		German		Er erwarb die amerikanische Staatsbürgerschaft.
spa		Spanish		Spanish		Yo ocupo todo el tiempo que puedo en leer.
eng		English		English		His life rests on her.


> **Recherche des phrases mal classées par Naive Bayes**

In [15]:
n_bad_max = 30
n_bad = 0
print("\tN°Ligne\tL. réelle\tPréd. Naive B.\t\tPhrase")
for i in range(len(df)):
    if (n_bad<n_bad_max):
        if (df['lan_code'].iloc[i] != lang_id_nb(df['sentence'].iloc[i])):
            n_bad +=1
            print(n_bad,'\t',i,'\t-',df['lan_code'].iloc[i],'\t\t'+lang_id_nb(df['sentence'].iloc[i]).ljust(12),'\t\t'+
                  df['sentence'].iloc[i]," (proba={:.2f}".format(max(clf_nb.predict_proba(create_BOW([df['sentence'].iloc[i]]))[0]))+")")

	N°Ligne	L. réelle	Préd. Naive B.		Phrase
1 	 0 	- eng 		English      		She is afraid of death.  (proba=1.00)
2 	 1 	- ita 		Italian      		Indovina cosa scelgo io.  (proba=1.00)
3 	 2 	- spa 		Spanish      		¿Puedo ayudarlo? "No, gracias. Solo estoy mirando."  (proba=1.00)
4 	 3 	- ita 		Italian      		Io non sono una fricchettona!  (proba=1.00)
5 	 4 	- deu 		German       		Es sind schon fast 10 Jahre vergangen, aber du bist unverändert schön.  (proba=1.00)
6 	 5 	- spa 		Spanish      		Creía que me quería.  (proba=1.00)
7 	 6 	- eng 		English      		This school sets high moral standards for pupils.  (proba=1.00)
8 	 7 	- eng 		English      		Man is judged by his courage, woman by her charm.  (proba=1.00)
9 	 8 	- fra 		French       		Je mange des pruneaux sucrés.  (proba=1.00)
10 	 9 	- fra 		French       		J'ai écrit une chanson pour toi.  (proba=1.00)
11 	 10 	- fra 		French       		Tom avait déjà rejoint son domicile.  (proba=1.00)
12 	 11 	- ita 		Italian      		Tom è una spia. 